In [1]:
import pandas as pd

In [32]:
model_input = pd.read_csv("C:/Users/ruchi/OneDrive/Ruchira-MAIN/Work/Stats_ML/model_input_data_v2.csv")

In [33]:
model_input.head()

,Unnamed: 0,adjusted_date,ticker,transcript,abvol_20d,abcallday_r1,abcallday_r5,abcallday_r20,abret_1d,abret_5d,r1d_direction,r5d_direction
0,2,2019-10-24,LRCX,Prepared Remarks:\nOperator\nGood day and welc...,0.032239,-0.010274,-0.011690,-0.037597,0.012659,0.010627,1,1
1,11,2021-08-26,SJM,"Prepared Remarks:\nOperator\nGood morning, and...",0.013258,-0.001548,-0.055643,-0.042188,-0.011765,-0.031104,0,0
2,19,2019-11-07,DXCM,Prepared Remarks:\nOperator\nWelcome to the De...,0.064064,-0.003093,-0.026555,-0.059416,0.014894,0.037209,1,1
3,24,2021-07-01,WBA,Prepared Remarks:\nOperator\nLadies and gentle...,0.020677,0.011084,-0.002744,-0.044386,-0.018729,-0.038514,0,0
4,25,2021-05-04,ZBRA,"Prepared Remarks:\nOperator\nGood day, and wel...",0.013530,-0.019974,-0.040430,-0.057039,0.024761,0.035372,1,1


In [34]:
import numpy as np
df = model_input.copy()

df = df.sort_values("adjusted_date").reset_index(drop=True)
dates = np.array(sorted(df["adjusted_date"].unique()))
n_dates = len(dates)

train_end = int(0.75 * n_dates)
val_end   = int(0.87 * n_dates)  # 65% + 10%

train_dates = dates[:train_end]
val_dates   = dates[train_end:val_end]
test_dates  = dates[val_end:]

train_df = df[df["adjusted_date"].isin(train_dates)].reset_index(drop=True)
val_df   = df[df["adjusted_date"].isin(val_dates)].reset_index(drop=True)
test_df  = df[df["adjusted_date"].isin(test_dates)].reset_index(drop=True)

print("Unique dates:", n_dates)
print("Rows:", len(train_df), len(val_df), len(test_df))
print("Train range:", train_dates[0], "→", train_dates[-1])
print("Val range:  ", val_dates[0], "→", val_dates[-1])
print("Test range: ", test_dates[0], "→", test_dates[-1])


Unique dates: 511
Rows: 2823 319 402
Train range: 2019-04-17 → 2022-02-10
Val range:   2022-02-11 → 2022-07-14
Test range:  2022-07-15 → 2022-12-01


In [35]:
y_train_1d = train_df["r1d_direction"]
y_val_1d   = val_df["r1d_direction"]
y_test_1d  = test_df["r1d_direction"]

y_train_5d = train_df["r5d_direction"]
y_val_5d   = val_df["r5d_direction"]
y_test_5d  = test_df["r5d_direction"]

features_base_train = train_df[["abvol_20d", "abcallday_r1", "abcallday_r5", "abcallday_r20"]]
features_base_val   = val_df[["abvol_20d", "abcallday_r1", "abcallday_r5", "abcallday_r20"]]
features_base_test  = test_df[["abvol_20d", "abcallday_r1", "abcallday_r5", "abcallday_r20"]]

# Baseline Model-0 (Random)

In [36]:
import random as rnd
rng = np.random.default_rng(seed=42)

In [37]:
p_5d_train=y_train_5d.sum()/len(y_train_5d)
p_5d_train

np.float64(0.49273822174991144)

In [38]:
p_pred_random_5d= rng.uniform(size=len(y_test_5d)) #generates probabilities randomly
y_pred_random_5d = (p_pred_random_5d >= 1-p_5d_train).astype(int) #converts probabilities to class labels based on 1-p_5d_train threshold

In [39]:
from sklearn.metrics import accuracy_score,roc_auc_score,confusion_matrix
accuracy_random_5d = accuracy_score(y_test_5d, y_pred_random_5d)
auc_random_5d=roc_auc_score(y_test_5d, p_pred_random_5d)

print("Random Model 5d - Accuracy:", accuracy_random_5d)
print("Random Model 5d - AUC:", auc_random_5d)

Random Model 5d - Accuracy: 0.5248756218905473
Random Model 5d - AUC: 0.531331921758419


# Baseline-1 Finance Data only

In [40]:
from sklearn.linear_model import LogisticRegression

In [42]:
C0=[0.01, 0.1, 1, 10, 100]
for C in C0:
    finance_model_5d = LogisticRegression(penalty='l2',C=C)
    finance_model_5d.fit(features_base_train, y_train_5d)
    val_probs_5d = finance_model_5d.predict_proba(features_base_val)[:, 1]
    print(f"C={C}: Coefficients={finance_model_5d.coef_}")
    val_auc_5d = roc_auc_score(y_val_5d, val_probs_5d)
    print(f"C={C}: Validation AUC={val_auc_5d}")


C=0.01: Coefficients=[[-0.00189309  0.00672585 -0.00508669 -0.13162844]]
C=0.01: Validation AUC=0.5079927553350657
C=0.1: Coefficients=[[-0.00724024  0.06998749 -0.00273093 -0.85833899]]
C=0.1: Validation AUC=0.5066146940703993
C=1: Coefficients=[[ 0.1146123   0.80152751  0.66117981 -2.08996934]]
C=1: Validation AUC=0.4955114575950862
C=10: Coefficients=[[ 1.23314941  3.61698572  0.97812679 -2.63965954]]
C=10: Validation AUC=0.48137648633750685
C=100: Coefficients=[[ 2.12236012  5.58736106  0.63867087 -2.71665006]]
C=100: Validation AUC=0.48129773997952596


In [43]:
finance_model_5d = LogisticRegression(penalty='l2',C=0.01)
finance_model_5d.fit(features_base_train, y_train_5d)
test_probs_5d = finance_model_5d.predict_proba(features_base_test)[:, 1]
test_auc_5d = roc_auc_score(y_test_5d, test_probs_5d)
print(f"Test AUC (C=0.01): {test_auc_5d}")

Test AUC (C=0.01): 0.5016132284734826


 A finance-only logistic regression model with C=0.01 using pre-call volatility and abnormal returns shows no predictive power for post-call drift. Validation AUC peaks at strong regularization (C=0.01) and the coefficients are almost zero for this value. Its performance is indistinguishable from the random model (AUC=0.5). 

# Baseline-2 TF-IDF 

In [44]:
X_train_text= train_df["transcript"]
X_val_text= val_df["transcript"]
X_test_text= test_df["transcript"]

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=50000, stop_words='english', ngram_range=(1,2),min_df=10,max_df=0.8,token_pattern=r"(?u)\b[a-zA-Z][a-zA-Z]+\b")
X_train_tfidf = vectorizer.fit_transform(X_train_text) 
X_val_tfidf = vectorizer.transform(X_val_text)
X_test_tfidf = vectorizer.transform(X_test_text)

In [61]:
Cs=[0.01, 0.1, 1, 10, 100]
best=None
for C in Cs:
    text_model_5d = LogisticRegression(penalty='l2',C=C,solver='liblinear',max_iter=2000)
    text_model_5d.fit(X_train_tfidf, y_train_5d)
    val_probs_5d = text_model_5d.predict_proba(X_val_tfidf)[:, 1]
    val_auc_5d = roc_auc_score(y_val_5d, val_probs_5d)
    print(f"C={C}: Validation AUC={val_auc_5d}")
    if best is None or val_auc_5d > best[0]:
        best = (val_auc_5d, C)

C=0.01: Validation AUC=0.571698558941649
C=0.1: Validation AUC=0.5752815182297819
C=1: Validation AUC=0.5731159933853059
C=10: Validation AUC=0.5634301913536499
C=100: Validation AUC=0.558508543979841


In [62]:
import numpy as np

# number of words to show
TOP_K = 20

# get feature names and coefficients
feature_names = np.array(vectorizer.get_feature_names_out())
coefs = text_model_5d.coef_[0]   # shape: (num_features,)

# top positive words (push prediction toward y=1)
top_pos_idx = np.argsort(coefs)[-TOP_K:][::-1]
top_pos = list(zip(feature_names[top_pos_idx], coefs[top_pos_idx]))

# top negative words (push prediction toward y=0)
top_neg_idx = np.argsort(coefs)[:TOP_K]
top_neg = list(zip(feature_names[top_neg_idx], coefs[top_neg_idx]))

print("Top positive words:")
for w, c in top_pos:
    print(f"{w:30s} {c:.4f}")

print("\nTop negative words:")
for w, c in top_neg:
    print(f"{w:30s} {c:.4f}")


Top positive words:
comes line                     7.4077
laura                          6.8223
commercial                     6.5540
cowen analyst                  5.7916
sewell                         5.7726
february                       5.6606
nice                           5.5863
school                         5.5788
gbs                            5.4897
case                           5.4205
proceed questions              5.4179
orders                         5.3459
january                        5.3392
winter                         5.3118
midstream                      5.2708
chairman board                 5.2664
officer director               5.2235
dose                           5.1876
martin                         5.0970
march                          5.0850

Top negative words:
july                           -8.1606
mts                            -6.7599
half year                      -5.9827
arena                          -5.9223
director president             -5.8868
acti

In [54]:
len(coefs)

50000